In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures

train = pd.read_csv('data/train.csv')

train_mod = train.copy(deep=True)
obj_columns = train_mod.select_dtypes(['object']).columns
train_mod[obj_columns] = train_mod[obj_columns].astype('category')
train_mod[obj_columns] = train_mod[obj_columns].apply(lambda x: x.cat.codes)
train_mod.fillna(train_mod.mean(), inplace=True)
x = train_mod[train_mod.columns[1:-1]].copy()
y = train_mod[train_mod.columns[-1]].copy()
fold = 10

def cross_validation(train_x, train_y, test_x, test_y, fold=fold):
    s = 0
    n = len(train_x[0])
    p = len(train_x[0].columns)
    assert n > p + 1
    for i in range(fold):
        r2 = LinearRegression().fit(train_x[i], train_y[i]).score(test_x[i], test_y[i])
        adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
        s += adjusted_r2 
    s /= fold
    return s

In [2]:
# question b
# 1460 samples in training set. 79 features. 
train[:3]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [3]:
cols = list(train)
for i in range(len(cols)):
    if train.dtypes[i] == object:
        print (cols[i], end=',')

MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,

In [4]:
# question c
# MSZoning, HouseStyle, OverallCond, 1stFlrSF, SaleType, YrSold, GarageArea, LotArea
matrix_plot = train_mod[['MSZoning', 'HouseStyle', 'OverallCond', '1stFlrSF', 'SaleType', 'YrSold', 'GarageArea', 'LotArea','SalePrice']]

In [5]:
pd.plotting.scatter_matrix(matrix_plot, alpha=0.2, figsize=(10, 10))
plt.savefig('scatterplot_matrix.png')
plt.clf()

<Figure size 720x720 with 0 Axes>

In [6]:
# question d
train_mod['const'] = 1
model = sm.OLS(y, x, missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     827.2
Date:                Fri, 05 Oct 2018   Prob (F-statistic):               0.00
Time:                        23:42:56   Log-Likelihood:                -17063.
No. Observations:                1460   AIC:                         3.428e+04
Df Residuals:                    1383   BIC:                         3.469e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
MSSubClass      -66.7349     42.369     -1.575      0.115    -149.848      16.379
MSZoning      -2086.2326   1456.684     -1.432      0.152   -4943.783     771.317
LotFrontage    -130.5456     47.148     -2.769      0.006    -223.035     -38.056
LotArea           0.4155      0.100      4.136      0.000       0.218       0.613
Street         3.432e+04   1.32e+04      2.601      0.009    8436.465    6.02e+04
Alley         -2820.0567   2447.859     -1.152      0.249   -7621.974    1981.861
LotShape       -758.4848    617.061     -1.229      0.219   -1968.962     451.993
LandContour    1752.0509   1259.326      1.391      0.164    -718.344    4222.446
Utilities     -4.669e+04    3.1e+04     -1.505      0.133   -1.08e+05    1.42e+04
LotConfig         1.0399    510.550      0.002      0.998   -1000.497    1002.576
LandSlope      4609.0156   3571.448      1.291      0.197   -2397.025    1.16e+04
Neighborhood    379.4213    146.097      2.597      0.010      92.826     666.017
Condition1     -659.7204    948.051     -0.696      0.487   -2519.495    1200.054
Condition2    -9405.4923   3129.979     -3.005      0.003   -1.55e+04   -3265.472
BldgType      -2936.0908   1399.454     -2.098      0.036   -5681.374    -190.808
HouseStyle     -672.4284    610.813     -1.101      0.271   -1870.649     525.793
OverallQual    1.023e+04   1108.093      9.230      0.000    8054.180    1.24e+04
OverallCond    5799.2123    978.926      5.924      0.000    3878.872    7719.553
YearBuilt       237.8243     73.730      3.226      0.001      93.190     382.459
YearRemodAdd     15.4811     63.903      0.242      0.809    -109.876     140.838
RoofStyle      1029.4787   1050.288      0.980      0.327   -1030.851    3089.808
RoofMatl       4563.4565   1403.476      3.252      0.001    1810.285    7316.628
Exterior1st   -1093.2665    489.005     -2.236      0.026   -2052.537    -133.996
Exterior2nd     673.7982    442.186      1.524      0.128    -193.629    1541.226
MasVnrType     3779.4336   1377.902      2.743      0.006    1076.430    6482.437
MasVnrArea       32.1559      5.657      5.684      0.000      21.058      43.254
ExterQual     -9895.1585   1837.880     -5.384      0.000   -1.35e+04   -6289.824
ExterCond       777.4286   1170.885      0.664      0.507   -1519.474    3074.331
Foundation       50.4207   1571.038      0.032      0.974   -3031.455    3132.296
BsmtQual      -7165.0447   1265.237     -5.663      0.000   -9647.037   -4683.053
BsmtCond       2332.2089   1223.008      1.907      0.057     -66.942    4731.359
BsmtExposure  -2923.3020    817.646     -3.575      0.000   -4527.263   -1319.341
BsmtFinType1   -325.5436    581.743     -0.560      0.576   -1466.739     815.651
BsmtFinSF1        7.8584      2.758      2.849      0.004       2.448      13.269
BsmtFinType2   2320.4046   1045.693      2.219   

In [7]:
# question e
# train test split and 10 fold data
train_x_total, test_x_total, train_y_total, test_y_total = train_test_split(x, y, test_size=0.2)
kf = KFold(n_splits=fold)
train_x = []
train_y = []
test_x = []
test_y = []
for train_index, test_index in kf.split(x):
    train_x.append(x.loc[train_index, :])
    train_y.append(y.iloc[train_index])
    test_x.append(x.loc[test_index, :])
    test_y.append(y.iloc[test_index])

In [8]:
# OLS
reg_OLS = LinearRegression().fit(train_x_total, train_y_total)
reg_OLS.score(test_x_total, test_y_total)

0.8741788780709241

In [9]:
# kNN
li = []
for i in range(1, 11):
    s = 0
    for j in range(fold):
        s += KNeighborsRegressor(n_neighbors=i).fit(train_x[j], train_y[j]).score(test_x[j], test_y[j])
    s /= 10
    li.append(s)
    print('mean accuracy for', i, 'neighbor:', s)
m = 0
for i, val in enumerate(li):
    if val > li[m]:
        m = i
print('max mean accuracy is', m + 1, 'neighbors:', li[m])

mean accuracy for 1 neighbor: 0.4992256478695805
mean accuracy for 2 neighbor: 0.6193329383408436
mean accuracy for 3 neighbor: 0.650989161690234
mean accuracy for 4 neighbor: 0.6584867452028489
mean accuracy for 5 neighbor: 0.6639139051366764
mean accuracy for 6 neighbor: 0.6599264835576119
mean accuracy for 7 neighbor: 0.6622698376637135
mean accuracy for 8 neighbor: 0.6633359332493693
mean accuracy for 9 neighbor: 0.6682815230526279
mean accuracy for 10 neighbor: 0.6675322846284222
max mean accuracy is 9 neighbors: 0.6682815230526279


In [10]:
# ridge regression
ridge = RidgeCV(cv=10).fit(train_x_total,train_y_total)
print("ridge alpha:", ridge.alpha_)
print(ridge.score(test_x_total, test_y_total))
print('ridge coef:', ridge.coef_)

ridge alpha: 0.1
0.8745248317774619
ridge coef: [-2.92923271e+01 -2.62748531e+03 -1.76320234e+02  3.74549811e-01
  4.00496248e+04 -2.18176286e+03 -5.95015454e+02  1.63734990e+03
 -3.64569929e+04 -1.73867906e+01  6.78671616e+03  3.51667548e+02
 -1.31078458e+03 -9.21315392e+03 -4.52305111e+03 -1.07205515e+03
  1.05116934e+04  6.14131860e+03  2.78554263e+02  3.50479014e+01
  1.01957936e+03  3.42358268e+03 -1.06010675e+03  5.29401909e+02
  4.37055419e+03  2.71018197e+01 -1.09411135e+04  2.31431469e+03
 -2.83449825e+01 -6.12783481e+03  2.51233245e+03 -3.23288195e+03
 -6.29294051e+02  5.39173016e+00  2.30880680e+03  1.25882558e+01
 -2.89535834e+00  1.50846306e+01 -9.32149754e+02 -3.82566861e+02
  1.14359433e+03 -4.59843260e+02  2.68921615e+01  2.60014162e+01
 -4.17915458e+01  1.11019832e+01  6.49748573e+03 -4.41652313e+02
  3.30344269e+03  2.68162636e+02 -3.83534224e+03 -1.83151014e+04
 -7.97448749e+03  5.44909254e+03  3.89303563e+03  9.21593163e+03
 -1.68607401e+03  2.04198257e+02 -1.224463

In [11]:
# LASSO
lasso = LassoCV(cv=10).fit(train_x_total, train_y_total)
print("lasso alpha:", lasso.alpha_)
print(lasso.score(test_x_total, test_y_total))

lasso alpha: 203529.7590739716
0.7953203533650579


In [12]:
for i in range(len(lasso.coef_)):
    if lasso.coef_[i] != 0:
        print(x.columns[i])

LotArea
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
TotalBsmtSF
1stFlrSF
2ndFlrSF
GrLivArea
GarageArea
WoodDeckSF
MiscVal


In [ ]:
# backward stepwise regression
attributes = list(x.columns)
best_score = cross_validation(train_x, train_y, test_x, test_y)
new_score = 0
best_attributes = attributes
flag = True
while flag:
    flag = False
    for i in range(len(attributes)):
        new_attributes = [_ for _ in attributes]
        del new_attributes[i]
        new_train_x = []
        for j in range(len(train_x)):
            new_train_x.append(train_x[j][new_attributes].copy())
        new_test_x = []
        for j in range(len(test_x)):
            new_test_x.append(test_x[j][new_attributes].copy())
        new_score = cross_validation(new_train_x, train_y, new_test_x, test_y)
        if new_score > best_score:
            best_score = new_score
            best_attributes = new_attributes
            flag = True
    attributes = best_attributes
print('final attributes number:', len(best_attributes))
print('best attributes:', best_attributes)
print('best adjusted r2:', best_score)
overall_best = best_attributes

In [ ]:
# forward stepwise regression
attributes = list(x.columns)
best_score = 0
new_score = 0
best_attributes = []
flag = True
while flag:
    flag = False
    temp = [_ for _ in best_attributes]
    add_attribute = None
    for i in range(len(attributes)):
        new_attributes = [_ for _ in temp]
        new_attributes.append(attributes[i])
        new_train_x = []
        for j in range(len(train_x)):
            new_train_x.append(train_x[j][new_attributes].copy())
        new_test_x = []
        for j in range(len(test_x)):
            new_test_x.append(test_x[j][new_attributes].copy())
        new_score = cross_validation(new_train_x, train_y, new_test_x, test_y)
        if new_score > best_score:
            best_score = new_score
            best_attributes = new_attributes
            add_attribute = i
            flag = True
    if flag:
        del attributes[add_attribute]
print('final attributes number:', len(best_attributes))
print('best attributes:', best_attributes)
print('best adjusted r2:', best_score)

In [ ]:
# question f
poly = PolynomialFeatures(2)
mx = poly.fit_transform(x)[:, 1:]

In [ ]:
feature_names = poly.get_feature_names(x.columns)
quadratic_x = pd.DataFrame(mx, columns=feature_names[1:])

In [ ]:
len(quadratic_x.columns)

In [ ]:
len(x.columns)

In [ ]:
# split into train and test
train_x_total, test_x_total, train_y_total, test_y_total = train_test_split(quadratic_x, y, test_size=0.2)

In [ ]:
# OLS
reg_OLS = LinearRegression().fit(train_x_total, train_y_total)
reg_OLS.score(test_x_total, test_y_total)

In [ ]:
kf = KFold(n_splits=fold)
train_x = []
train_y = []
test_x = []
test_y = []
for train_index, test_index in kf.split(quadratic_x):
    train_x.append(quadratic_x.loc[train_index, :])
    train_y.append(y.iloc[train_index])
    test_x.append(quadratic_x.loc[test_index, :])
    test_y.append(y.iloc[test_index])

In [ ]:
# kNN
li = []
for i in range(1, 11):
    s = 0
    for j in range(fold):
        s += KNeighborsRegressor(n_neighbors=i).fit(train_x[j], train_y[j]).score(test_x[j], test_y[j])
    s /= 10
    li.append(s)
    print('mean accuracy for', i, 'neighbor:', s)
m = 0
for i, val in enumerate(li):
    if val > li[m]:
        m = i
print('max mean accuracy is', m + 1, 'neighbors:', li[m])

In [ ]:
# ridge regression
ridge = RidgeCV(cv=10).fit(train_x_total,train_y_total)
print("ridge alpha:", ridge.alpha_)
print(ridge.score(test_x_total, test_y_total))
print('ridge coef:', ridge.coef_)

In [ ]:
# LASSO
lasso = LassoCV(cv=10).fit(train_x_total, train_y_total)
print("lasso alpha:", lasso.alpha_)
print(lasso.score(test_x_total, test_y_total))
print('lasso coef:', lasso.coef_)

In [ ]:
# Too slow to run
# backward stepwise regression
# attributes = list(x.columns)
# best_score = cross_validation(train_x, train_y, test_x, test_y)
# new_score = 0
# best_attributes = attributes
# flag = True
# while flag:
#     flag = False
#     for i in range(len(attributes)):
#         new_attributes = [_ for _ in attributes]
#         del new_attributes[i]
#         new_train_x = []
#         for j in range(len(train_x)):
#             new_train_x.append(train_x[j][new_attributes].copy())
#         new_test_x = []
#         for j in range(len(test_x)):
#             new_test_x.append(test_x[j][new_attributes].copy())
#         new_score = cross_validation(new_train_x, train_y, new_test_x, test_y)
#         if new_score > best_score:
#             best_score = new_score
#             best_attributes = new_attributes
#             flag = True
#     attributes = best_attributes
# print('final attributes number:', len(best_attributes))
# print('best attributes:', best_attributes)
# print('best adjusted r2:', best_score)

In [ ]:
# Too slow to run
# forward stepwise regression
# attributes = list(quadratic_x.columns)
# best_score = 0
# new_score = 0
# best_attributes = []
# flag = True
# while flag:
#     flag = False
#     temp = [_ for _ in best_attributes]
#     add_attribute = None
#     for i in range(len(attributes)):
#         new_attributes = [_ for _ in temp]
#         new_attributes.append(attributes[i])
#         new_train_x = []
#         for j in range(len(train_x)):
#             new_train_x.append(train_x[j][new_attributes].copy())
#         new_test_x = []
#         for j in range(len(test_x)):
#             new_test_x.append(test_x[j][new_attributes].copy())
#         new_score = cross_validation(new_train_x, train_y, new_test_x, test_y)
#         if new_score > best_score:
#             best_score = new_score
#             best_attributes = new_attributes
#             add_attribute = i
#             flag = True
#     if flag:
#         del attributes[add_attribute]
# print('final attributes number:', len(best_attributes))
# print('best attributes:', best_attributes)
# print('best adjusted r2:', best_score)

In [ ]:
# question h
test = pd.read_csv('data/test.csv')
test_mod = test.copy(deep=True)
obj_columns = test_mod.select_dtypes(['object']).columns
test_mod[obj_columns] = test_mod[obj_columns].astype('category')
test_mod[obj_columns] = test_mod[obj_columns].apply(lambda x: x.cat.codes)
test_mod.fillna(test_mod.mean(), inplace=True)
first_column = test_mod[test_mod.columns[0]]
# test_mod = test_mod[test_mod.columns[1:]]
test_mod = test_mod[overall_best]
new_linear = LinearRegression().fit(x[overall_best], y)
prediction = new_linear.predict(test_mod)

# reg_OLS = LinearRegression().fit(x, y)
# reg_OLS.score(test_x_total, test_y_total)

In [ ]:
reg_OLS = LinearRegression().fit(x, y)
prediction = reg_OLS.predict(test_mod)

In [ ]:
f = open('predict_backward_stepwise.csv', 'w')
f.write('Id,SalePrice\n')
for i in range(len(first_column)):
    f.write(str(first_column[i])+','+str(prediction[i])+'\n')
f.close()